In [1]:
import matplotlib
matplotlib.use('tkAgg')
import matplotlib.pyplot as plt
import glob
import sys
from matplotlib.ticker import MultipleLocator
import json
import os
import pandas as pd
import numpy as np
import pdb
from IPython.display import Image

In [7]:
batches = [1, 2, 4, 8]
models = ['base']
sequences = [128, 384]
precisions = ['amp', 'noamp']
LABELS = ['FP16', 'FP32']
colors = ['orangered', 'deepskyblue']

fig1, axs1 = plt.subplots(1, 2, gridspec_kw={'hspace': 0.2, 'wspace': 0.23, 'bottom': 0.2, 'top': 0.83, 'right':0.995, 'left':0.15}, figsize=(7,3))
fig2, axs2 = plt.subplots(1, 2, gridspec_kw={'hspace': 0.2, 'wspace': 0.23, 'bottom': 0.2, 'top': 0.83, 'right':0.995, 'left':0.15}, figsize=(7,3))
fig3, axs3 = plt.subplots(1, 2, gridspec_kw={'hspace': 0.2, 'wspace': 0.23, 'bottom': 0.2, 'top': 0.83, 'right':0.995, 'left':0.15}, figsize=(7,3))
fig4, axs4 = plt.subplots(1, 2, gridspec_kw={'hspace': 0.2, 'wspace': 0.23, 'bottom': 0.2, 'top': 0.83, 'right':0.995, 'left':0.15}, figsize=(7,3))
origin = 'squad_origin.log'
trt = 'squad_trt.log'
x = np.arange(len(batches))
width=0.2
bar_offset = [-0.5, 0.5]

In [8]:
for model in models:
    i = models.index(model)
    for seq in sequences:
        j = sequences.index(seq)
        for axs in [axs1, axs2, axs3, axs4]:
            axs[j].set_title(f'Sequence Length {seq}', fontsize=14)
        for pre in precisions:
            qps, mean, tail, tail99 = [], [], [], []
            ind = precisions.index(pre)
            for batch in batches:
                key = f'{model} {seq} {batch} --{pre}'
                with open(origin) as file:
                    lines_origin = file.readlines()
                    lines_origin = [line.rstrip() for line in lines_origin]
                for line in lines_origin:
                    if key in line:
                        meas = line.split(' ')        
                        qps_origin = float(meas[4])
                        mean_lat_origin = float(meas[5])
                        tail_lat_origin = float(meas[-2]) # 95th tail
                        tail_99_origin = float(meas[-1])
                        break
                with open(trt) as file:
                    lines_trt = file.readlines()
                    lines_trt = [line.rstrip() for line in lines_trt]
                for line in lines_trt:
                    if key in line:
                        meas = line.split(' ')        
                        qps_trt = float(meas[4])
                        mean_lat_trt = float(meas[5])
                        tail_lat_trt = float(meas[-2]) # 95th tail
                        tail_99_trt = float(meas[-1])
                        break
                qps.append(qps_trt / qps_origin)
                mean.append(mean_lat_trt / mean_lat_origin)
                tail.append(tail_lat_trt / tail_lat_origin)
                tail99.append(tail_99_trt / tail_99_origin)
            axs1[j].bar(x+width*bar_offset[ind], qps, width=width, color=colors[ind], label=LABELS[ind], 
                edgecolor='black', zorder=3)
            axs2[j].bar(x+width*bar_offset[ind], mean, width=width, color=colors[ind], label=LABELS[ind], 
                edgecolor='black', zorder=3)
            axs3[j].bar(x+width*bar_offset[ind], tail, width=width, color=colors[ind], label=LABELS[ind], 
                edgecolor='black', zorder=3)     
            axs4[j].bar(x+width*bar_offset[ind], tail99, width=width, color=colors[ind], label=LABELS[ind], 
                edgecolor='black', zorder=3)                 
for axs in [axs1, axs2, axs3, axs4]:
    axs[1].legend(fontsize=12)
    for ax in axs:
        ax.set_xticks(x)
        ax.set_xticklabels(batches, fontsize=14)
        ax.grid(which='major', axis='y', ls='dashed')
        ax.tick_params(axis='y', labelsize=14)
        ax.set_xlabel('Batch Size', fontsize=14)
axs1[0].set_ylabel('TRT Inference\nThroughput (Norm.)', fontsize=14)
axs2[0].set_ylabel('TRT Mean\nLatency (Norm.)', fontsize=14)
axs3[0].set_ylabel('TRT 95th Tail\nLatency (Norm.)', fontsize=14)
axs4[0].set_ylabel('TRT 99th Tail\nLatency (Norm.)', fontsize=14)
#axs.set_ylim(0, 1.8)        

Text(0, 0.5, 'TRT 99th Tail\nLatency (Norm.)')

In [4]:
fig1.show()
fig2.show()
fig3.show()
fig4.show()

In [9]:
fig1.savefig('plots/qps.png')
fig2.savefig('plots/mean.png')
fig3.savefig('plots/tail95.png')
fig4.savefig('plots/tail99.png')